## Linear Programming

### Sets

There are several suppliers and several segments/areas, due to which we create set of suppliers and areas. 

- Let $S$ be a set of suppliers.
- Let $A$ be a set of areas. 

### Parameters

- For each area $a \in A$ and each supplier $s \in S$ we are given the cost $CQA_{a,s}$ (per ton) for material A when taken from X.
- For each area $a \in A$ and each supplier $s \in S$ we are given the cost $CGA_{a,s}$ (per ton) for Granular A when taken from Y. 
- For each area $a \in A$ and each supplier $s \in S$ we are given the cost $CQB_{a,s}$ (per ton)for Granular B when taken from X. 
- For each area $a \in A$ we are given demand of material A $DA_a$. 
- For each area $a \in A$ we are given demand of material B $DB_a$. 


Note: The cost includes the cost of hauling the material plus the cost of shipping to the designated area. 

### Variables

- For each area $a \in A$ and each supplier $s \in S$ we introduce continuous non-negative variables $XQA_{a,s}$, $XGA_{a,s}$, $XQB_{a,s}$.

- $XQA_{a,s}$ describes how much we procure granular A for area $a \in A$ from suppliers $s \in S$ from X. 
- $XGA_{a,s}$ describes how much we procure granular A for area $a \in A$ from supplier $s \in S$ from Y.
- $XQB_{a,s}$ describes how much we procure granular B for area $a \in A$ from supplier $s \in S$ from X.
- $XQA_{a,s}, XGA_{a,s}, XQB_{a,s} \geq 0 ~\forall a \in A, \forall s \in S$

### Objective

$$min \left( \sum_{s \in S} \sum_{a \in A} XQA_{a,s} \cdot CQA_{a,s} + XGA_{a,s} \cdot CGA_{a,s}+ XQB_{a,s}\cdot CQB_{a,s}\right)$$

### Constraints

- No supplier would be awarded more than 50% of the total supply contract.$$ \sum_{a \in A} XQA_{s}+ XGA_{s}+ XQB_{s} <=0.5 * \sum_{a \in A}DA_{a} + DB_{a} ~\forall s \in S$$

- Supplier 2 could only supply a maximum of 300,000 tons of materil A from X.$$\sum_{a \in A}XAG_{a,2}<=300,000 $$


- Supplier 4’s total maximum capacity is about 1,000,000 tons.$$\sum_{a \in A} XQA_{a,4}+XGA_{a,4}+XQB_{a,4}<=1,000,000 $$

- Supplier 7’s total maximum delivery capacity was 300,000 tons.$$\sum_{a \in A} XQA_{a,7}+XQB_{a,7}<=300,000 $$

- The quantity procured of material A should satisfy the demand of material A.$$\sum_{s \in S} XQA_{a,s}+XGA_{a,s}= \sum_{a \in A}DA_a  ~\forall a \in A $$

- The quantity procured of material B should satisfy the demand of material B. $$\sum_{s \in S} XQB_{a,s}= \sum_{a \in A} DB_a ~\forall a \in A $$

### Implementation

In [ ]:
import numpy as np
import pandas as pd
import gurobipy as GRB

In [ ]:
def ReadInstance(CGA_csv,CQA_csv,CQB_csv,DA_csv,DB_csv):
    S=[]
    for i in CQA_csv.columns[1:]:
        if i not in S:
            S.append(i)
    A=[]
    for i in CQA_csv['Area']:
        if i not in A:
            A.append(i)
    
    # Parameters
    CQA_csv_np=CQA_csv.to_numpy()
    CQA={}
    count=0
    for i in A:
        CQA_1={}
        for j in S:
            CQA_1[j]=CQA_csv[j][count]
        CQA[i]=CQA_1
        count+=1
    
    CGA_csv_np=CGA_csv.to_numpy()
    CGA={}
    count=0
    for i in A:
        CGA_1={}
        for j in S:
            CGA_1[j]=CGA_csv[j][count]
        CGA[i]=CGA_1
        count+=1 
        
    CQB_csv_np=CQB_csv.to_numpy()
    CQB={}
    count=0
    for i in A:
        CQB_1={}
        for j in S:
            CQB_1[j]=CQB_csv[j][count]
        CQB[i]=CQB_1
        count+=1 
    
    DA_np = DA_csv.to_numpy()
    DA={}
    for i in range(0,len(A)):
        DA[A[i]]=DA_np[i][1]
        
    DB_np = DB_csv.to_numpy()
    DB={}
    for i in range(0,len(A)):
        DB[A[i]]=DB_np[i][1]
    
    return S,A,CQA,CGA,CQB,DA,DB

In [ ]:
S,A,CQA,CGA,CQB,DA,DB= ReadInstance(CGA_csv,CQA_csv,CQB_csv,DA_csv,DB_csv)

In [ ]:
m1 = GRB.Model('option1')

# material A procured from X
XQA = m1.addVars(A, S, name="material A procured from X") 
# material A procured from Y
XGA = m1.addVars(A, S, name="material A procured from Y") 
# material B procured from X
XQB = m1.addVars(A, S, name="material B procured from X") 

In [ ]:
## Objective Function
objective1=(GRB.quicksum(XGA[a,s]*CGA[a][s]+ XQA[a,s]*CQA[a][s]+XQB[a,s]*CQB[a][s] 
                            for a in A for s in S))

m1.setObjective(objective1)

In [ ]:
# Constraints
# No supplier would be awarded more than 50% of the total supply contract.
m1.addConstrs((GRB.quicksum(XQA[a,s] + XGA[a,s] + XQB[a,s] for a in A)
                 <=  0.5  * (DA_csv['Quantity (tn)'].sum()+DB_csv['Quantity (tn)'].sum()) for s in S),name="HalfSupply")

In [ ]:
# Supplier 2 could only supply a maximum of 300,000 tons of A
m1.addConstr((GRB.quicksum(XGA[a,'Supplier 2'] for a in A) <= 300000 ),name="Supplier2Constraint")

In [ ]:
# Supplier 4’s total maximum capacity is about 1,000,000 tons.
m1.addConstr((GRB.quicksum(XGA[a,'Supplier 4']+XQA[a, 'Supplier 4']+XQB[a, 'Supplier 4'] for a in A)<= 1000000 ),  
                 name="Supplier4Constraint")

In [ ]:
# Supplier 7’s total maximum delivery capacity was 300,000 tons.
m1.addConstr(GRB.quicksum(XGA[a,'Supplier 7']+XQA[a,'Supplier 7']+XQB[a,'Supplier 7'] for a in A) <= 300000,  
                 name="Supplier7Constraint")

In [ ]:
# The quantity procured of A should satisfy the demand of A.
m1.addConstrs((GRB.quicksum(XGA[a,s]+XQA[a,s] for s in S) == DA[a] for a in A),  
                 name="DemandA")# The quantity procured of A should satisfy the demand of A.
m1.addConstrs((GRB.quicksum(XGA[a,s]+XQA[a,s] for s in S) == DA[a] for a in A),  
                 name="DemandA")

In [ ]:
# The quantity procured of B should satisfy the demand of B.
m1.addConstrs((GRB.quicksum(XQB[a,s] for s in S) == DB[a] for a in A),  
                 name="DemandB")

### Results 

In [ ]:
# Compute optimal solution
m1.optimize()